In [2]:
"""
Optimized k means
"""
import numpy as np
from collections import deque
M = np.random.randint(0, 500, (10000, 1, 2))

def weighted_sample(weights):
    """
    Sample a weighted probability distribution
    returns an index
    """
    total_w = weights / np.sum(weights)
    # print(total_w)
    sample_val = np.random.uniform(0,1)
    # print(sample_val)
    for idx, w in enumerate(total_w):

        sample_val -= w
        
        if sample_val <= 0:
            return idx
    return len(weights) - 1

def distance(k, m):
    return np.sqrt(np.sum(np.power(k - m, 2)))


def assign_clusters(data, centroids):
    clusters = [[] for _ in centroids]
    min_dist = float("inf")
    nn = None
    pdist = None
    for img in data:
        min_dist = float("inf")
        nn = None
        for idx, ctr in enumerate(centroids):
            if np.array_equal(img, ctr):
                continue
            pdist = distance(img, ctr)
            if pdist < min_dist:
                min_dist = pdist
                nn = idx
        if nn != None:
            clusters[nn].append(img)
    # ret_clus = []
    for i in range(len(clusters)):
        if not len(clusters[i]):
            continue
        
        clusters[i] = np.stack([j for j in clusters[i]])


    return clusters


def update_centroids(clusters):
    return np.array([np.mean(cluster, axis=0) if len(cluster) else np.zeros_like(cluster[0]) for cluster in clusters])

def kmeanspp(M, k, centroids, not_chosen, chosen):
    """
    Compute a probably-better-than-random set of k centroids given an arr
    """
    
    for _ in range(k):
        weights = np.zeros(len(not_chosen))
        D = lambda ck, m: np.sqrt(
            np.sum(np.array([np.power(i, 2) for i in (ck - m).flatten()]))
        )
        for idx, mdx in enumerate(not_chosen):
            m = M[mdx]
            min_dist = float("inf")
            for ck in centroids:
                min_dist = min(min_dist, D(ck, m))
            weights[idx] = np.power(min_dist, 2)

        selected_point = weighted_sample(weights)
        centroids.append(M[not_chosen[selected_point]])

        chosen.add(not_chosen[selected_point])
        not_chosen.remove(not_chosen[selected_point])

    centroids = np.array(centroids)
    return centroids


def kmeans(M, k, max_iters=100):
    start_center = np.random.randint(M.shape[0])
    centroids = [M[start_center]]

    not_chosen = deque(i for i in range(len(M)) if i != start_center)
    chosen = {start_center}

    centroids = kmeanspp(M, k, centroids, not_chosen, chosen)

    for _ in range(max_iters):
        clusters = assign_clusters(M, centroids)
        new_centroids = update_centroids(clusters)
        if np.array_equal(centroids, new_centroids):
            break

        centroids = new_centroids

    return clusters, centroids

k = 15
clusters, centroids = kmeans(M, k)
centers = []
reclus = []
# print(clusters)
for idx,k in enumerate(clusters):
    if not len(k):
        continue
    reclus.append(np.array(clusters[idx]))
    centers.append(centroids[idx])

centers = np.array(centers)





Writing optimized_kmeans.py


In [3]:
"""
original k means
"""
import numpy as np
from collections import deque

def weighted_sample(weights):
    """
    Sample a weighted probability distribution
    returns an index
    """
    total_w = weights / np.sum(weights)
    sample_val = np.random.uniform(0, 1)
    for idx, w in enumerate(total_w):
        sample_val -= w
        if sample_val <= 0:
            return idx
    return len(weights) - 1

M = np.random.randint(0, 500, (10000, 1, 2))

# initialize not chosen

def distance(k,m):
    """
    Distance function between k,m
    """
    return np.sqrt(np.sum(np.array([np.power(i, 2) for i in (k - m).flatten()])))
    # return D(k,m)

def assign_clusters(data, centroids):
    """
    Assign data to clusters using distance from nearest centroid
    """
    clusters = [[] for _ in centroids]
    for img in data:
        min_dist = float("inf")
        nn = None
        for idx, ctr in enumerate(centroids):
            if np.array_equal(img, ctr):
                continue
            pdist = distance(img, ctr)
            if pdist < min_dist:
                min_dist = pdist
                nn = idx
        if nn != None:
            clusters[nn].append(img)
    ret_clus = []
    for i in range(len(clusters)):
        if not len(clusters[i]):
            continue
        
        clusters[i] = np.stack([j for j in clusters[i]])

    return clusters

def update_centroids(clusters, old_centroids):
    """
    Recenter the old centroids to be the mean of the new clusters
    """
    centroids = [np.zeros((1, 2)) for _ in clusters]
    for idx, cluster in enumerate(clusters):
        rep = old_centroids[idx]
        if len(cluster) and len(cluster[0]):
            rep = np.mean(cluster, axis=0)
        centroids[idx] = rep
    return np.array(centroids)

def kmeanspp(M, k, centroids, not_chosen, chosen):
    """
    Compute a probably-better-than-random set of k centroids given an arr
    """
    for _ in range(k):
        weights = np.zeros(len(not_chosen))
        D = lambda ck, m: np.sqrt(
            np.sum(np.array([np.power(i, 2) for i in (ck - m).flatten()]))
        )
        for idx, mdx in enumerate(not_chosen):
            m = M[mdx]
            min_dist = float("inf")
            for ck in centroids:
                min_dist = min(min_dist, D(ck, m))
            weights[idx] = np.power(min_dist, 2)

        selected_point = weighted_sample(weights)
        centroids.append(M[not_chosen[selected_point]])

        chosen.add(not_chosen[selected_point])
        not_chosen.remove(not_chosen[selected_point])

    centroids = np.array(centroids)
    return centroids

def kmeans(M, k, max_iters=100):
    """
    Implementation of k means algorithm
    """
    start_center = np.random.randint(M.shape[0])

    centroids = [M[start_center]]

    not_chosen = deque()
    chosen = set()
    chosen.add(start_center)

    for i in range(len(M)):
        if i == start_center:
            continue
        not_chosen.append(i)

    # initialize centroids using kmeans++
    centroids = kmeanspp(M, k, centroids, not_chosen, chosen)
    # centroids = M[np.random.choice(1000, k, replace=False)]

    for _ in range(max_iters):
        clusters = assign_clusters(M, centroids)
        new_centroids = update_centroids(clusters, centroids)
        if np.array_equal(centroids,new_centroids):
            break

        centroids = new_centroids
    return clusters, centroids



k = 15
clusters, centroids = kmeans(M, k)
centers = []
reclus = []
# print(clusters)
for idx,k in enumerate(clusters):
    if not len(k):
        continue
    reclus.append(np.array(clusters[idx]))
    centers.append(centroids[idx])

centers = np.array(centers)

Writing original_kmeans.py


In [4]:

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def graham_scan_convex_hull(points):
    def cross_product(p, q, r):
        return (q[0] - p[0]) * (r[1] - p[1]) - (r[0] - p[0]) * (q[1] - p[1])

    def polar_angle(p, q):
        return np.arctan2(q[1] - p[1], q[0] - p[0])

    # Find the point with the lowest y-coordinate (and leftmost if tied)
    start_point = min(points, key=lambda p: (p[1], p[0]))

    # Sort the points based on polar angles with respect to the start point
    sorted_points = sorted(points, key=lambda p: (polar_angle(start_point, p), -p[1], p[0]))

    # Initialize the convex hull with the first three sorted points
    convex_hull = [sorted_points[0], sorted_points[1], sorted_points[2]]

    # Iterate through the sorted points and construct the convex hull
    for point in sorted_points[3:]:
        while len(convex_hull) >= 2 and cross_product(convex_hull[-2], convex_hull[-1], point) <= 0:
            convex_hull.pop()
        convex_hull.append(point)

    return convex_hull

# Reshape the array to (100, 2) for plotting
colors = list(set(mcolors.CSS4_COLORS))[::4]
# print(colors)
reshaped_centers = centers.reshape(-1, 2)
cx_coords = reshaped_centers[:, 0]
cy_coords = reshaped_centers[:, 1]

for i,M in enumerate(reclus):

  reshaped_data = M.reshape(-1, 2)
  print(reshaped_data.shape)
    # Compute the convex hull using Graham Scan
  convex_hull = graham_scan_convex_hull(reshaped_data)
  convex_hull.append(convex_hull[0])


  # Convert the convex hull points to a numpy array for visualization
  convex_hull_array = np.array(convex_hull)
  

  # Plot the original points and the convex hull
  # plt.scatter(reshaped_data[:, 0], reshaped_data[:, 1], c='blue', label='Points')
  plt.plot(convex_hull_array[:, 0], convex_hull_array[:, 1], c=colors[i], marker='o', label='Convex Hull')

  x_coords = reshaped_data[:, 0]
  y_coords = reshaped_data[:, 1]
  plt.scatter(x_coords, y_coords, marker="o", color=colors[i], s=2)
  plt.scatter(cx_coords[i], cy_coords[i], marker="x", color=colors[i], s=30)

# print(clusters.shape)
# Extract x and y coordinates




# Create a scatter plot



# Add labels and title
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.title("Scatter Plot of Data Points")

# Display the plot
plt.show()

Overwriting clusterviz.py
